# Asignaciones financieras


## Datos de proyecto en análisis


In [2]:
proyecto = "callacalla"
mes = 5
anio = 2025

## Librerias necesarias


In [3]:
import pandas as pd
import os
import pprint
import pickle
import json
import xlsxwriter
from xlsxwriter.utility import xl_range, xl_rowcol_to_cell
import re
from typing import Dict, List, Any
import itertools
from pathlib import Path
from dotenv import load_dotenv, find_dotenv
from google.oauth2 import service_account
from google.cloud import firestore
import excel2img
from dotenv import load_dotenv, find_dotenv
from openpyxl import load_workbook
from openpyxl.utils import get_column_letter
import win32com.client as win32

## Funciones utiles


### Calculadora de costo total


In [4]:
from typing import TypedDict


class CostoIngenieriaResult(TypedDict):
    costo_directo: float
    gastos_generales: float
    utilidad: float
    subtotal: float
    igv: float
    total: float


def calculadora_costo_total(costo_directo: float) -> CostoIngenieriaResult:
    """
    Calcula el costo total de un proyecto de ingeniería civil en soles peruanos.

    Args:
        costo_directo (float): Costo directo del proyecto

    Returns:
        CostoIngenieriaResult: Diccionario con todos los valores calculados
    """
    # Calcular gastos generales (10% del costo directo)
    gastos_generales = round(costo_directo * 0.10, 2)

    # Calcular utilidad (5% del costo directo)
    utilidad = round(costo_directo * 0.05, 2)

    # Calcular subtotal
    subtotal = round(costo_directo + gastos_generales + utilidad, 2)

    # Calcular IGV (18% del subtotal)
    igv = round(subtotal * 0.18, 2)

    # Calcular total
    total = round(subtotal + igv, 2)

    return {
        "costo_directo": round(costo_directo, 2),
        "gastos_generales": gastos_generales,
        "utilidad": utilidad,
        "subtotal": subtotal,
        "igv": igv,
        "total": total,
    }

### Multiplicar el precio unitario actualizado por la carga trabajo


In [5]:
def calcular_costos_unitarios(
    dict_precios_unitarios_actualizados, cargas_trabajo_contratista
):
    """
    Multiplica cada carga de trabajo del contratista por su precio unitario
    correspondiente.

    Args:
        dict_precios_unitarios_actualizados (dict): mapea código → precio unitario.
        cargas_trabajo_contratista (dict): mapea código → carga de trabajo.

    Returns:
        dict: mapea código → precio total (precio unitario * carga).

    Raises:
        KeyError: si alguna clave de cargas_trabajo_contratista no existe en
                  dict_precios_unitarios_actualizados.
    """
    # Comprobar que no falte ninguna clave
    faltantes = set(cargas_trabajo_contratista) - set(
        dict_precios_unitarios_actualizados
    )
    if faltantes:
        raise KeyError(f"Faltan precios unitarios para las claves: {faltantes}")

    # Generar el diccionario resultado
    resultado = {
        clave: dict_precios_unitarios_actualizados[clave]
        * cargas_trabajo_contratista[clave]
        for clave in cargas_trabajo_contratista
    }
    return resultado

### Fusionar diccionarios

In [6]:
def fusionar_diccionarios(diccionario_de_diccionarios):
    """
    Fusiona diccionarios separados en un diccionario unificado.
    Solo incluye claves donde ambos valores sean diferentes de cero.
    Usa dinámicamente las claves del diccionario de entrada.
    
    Args:
        diccionario_de_diccionarios (dict): Diccionario con estructura:
            {
                "nombre_campo1": {clave: valor, ...},
                "nombre_campo2": {clave: valor, ...}
            }
    
    Returns:
        dict: Diccionario fusionado con estructura {clave: {nombre_campo1: valor, nombre_campo2: valor}}
    """
    # Obtener las claves (nombres de los campos) del diccionario principal
    nombres_campos = list(diccionario_de_diccionarios.keys())
    
    if len(nombres_campos) != 2:
        raise ValueError("El diccionario debe contener exactamente 2 campos")
    
    campo1_nombre = nombres_campos[0]
    campo2_nombre = nombres_campos[1]
    
    campo1_datos = diccionario_de_diccionarios[campo1_nombre]
    campo2_datos = diccionario_de_diccionarios[campo2_nombre]
    
    fusionado = {}
    
    # Obtener todas las claves únicas de ambos diccionarios
    todas_las_claves = set(campo1_datos.keys()) | set(campo2_datos.keys())
    
    for clave in todas_las_claves:
        valor1 = campo1_datos.get(clave, 0)
        valor2 = campo2_datos.get(clave, 0)
        
        # Solo agregar si ambos valores son diferentes de cero
        if valor1 != 0 and valor2 != 0:
            fusionado[clave] = {
                campo1_nombre: valor1,
                campo2_nombre: valor2
            }
    
    return fusionado

### Formatear progresiva

In [7]:
def formatear_progresiva(distancia, decimales=0):
    """
    Convierte una distancia en metros a notación de progresiva.
    
    Parámetros:
    - distancia: int o float, la distancia en metros.
    - decimales: int, número de decimales a mostrar en la parte de los metros.
    
    Retorna:
    - str: progresiva en formato 'K+XXX' con los decimales indicados.
    """
    if not isinstance(distancia, (int, float)):
        raise ValueError("La distancia debe ser un número (int o float).")
    if not isinstance(decimales, int) or decimales < 0:
        raise ValueError("Los decimales deben ser un entero no negativo.")
    
    km = int(distancia) // 1000
    metros = distancia - (km * 1000)
    
    formato_metros = f"{metros:0.{decimales}f}".zfill(3 + (1 if decimales > 0 else 0) + decimales)
    return f"{km}+{formato_metros}"

### Calcular avance

In [8]:
def calcular_avance(programado, ejecutado):
    """
    Calcula el porcentaje de avance basado en lo programado y lo ejecutado.

    Si lo programado es 0 y lo ejecutado es mayor a 0, devuelve 'ejecución adelantada'.
    Si ambos son 0, devuelve 0.0.
    En cualquier otro caso, devuelve el porcentaje (0-1) como float redondeado a 2 decimales.
    """
    if programado == 0:
        if ejecutado > 0:
            return "Ejec. adelantada"
        else:
            return 0.0
    else:
        porcentaje = (ejecutado / programado) 
        return porcentaje

### Ordenar por centena

In [9]:
def ordenar_por_centena(data: Any) -> List[Dict[str, Any]]:
    """
    Toma un dict (o un JSON en formato str) cuyas claves acaban en número
    y devuelve una lista de dicts {'key':…, 'value':…} ordenada por ese número.
    Compatible con Firestore (to_dict()) y JSON.
    """
    # Si viene como cadena JSON, lo convertimos
    if isinstance(data, str):
        data = json.loads(data)
    
    # Aseguramos que sea dict
    if not isinstance(data, dict):
        raise ValueError("Se esperaba un diccionario o un string JSON que represente un diccionario.")

    pattern = re.compile(r'(\d+)$')

    def obtener_clave_numerica(item):
        clave = str(item[0])  # Convertimos clave a string en caso no lo sea
        match = pattern.search(clave)
        if match:
            return int(match.group(1))
        else:
            return float('inf')  # Opcional: claves sin número al final se van al final

    sorted_items = sorted(data.items(), key=obtener_clave_numerica)

    return [{"key": k, "value": v} for k, v in sorted_items]

### Clave grupo

In [10]:
def clave_grupo(item):
    return int(item['key'][2:]) // 100  # 101→1, 201→2, etc.

## Carga de datos


### Firebase


In [11]:
# 1. Busca el .env en el directorio actual o en cualquiera de los padres
dotenv_path = find_dotenv()
if not dotenv_path:
    raise FileNotFoundError(
        "No se encontró ningún archivo .env en este directorio ni en sus padres."
    )
load_dotenv(dotenv_path)

# 2. Define el root del proyecto como la carpeta que contiene el .env
project_root = Path(dotenv_path).parent

# 3. Obtén la ruta relativa de las credenciales desde la variable de entorno
rel_cred_path = os.getenv("FIRESTORE_CREDENTIALS")
if not rel_cred_path:
    raise RuntimeError("No existe la variable FIRESTORE_CREDENTIALS en el .env")

# 4. Construye la ruta absoluta al JSON
cred_path = Path(rel_cred_path)
if not cred_path.is_absolute():
    cred_path = (project_root / cred_path).resolve()

if not cred_path.exists():
    raise FileNotFoundError(f"No existe el archivo de credenciales en: {cred_path}")

# 5. Carga las credenciales y crea el cliente de Firestore
credentials = service_account.Credentials.from_service_account_file(str(cred_path))
client = firestore.Client(credentials=credentials, project=credentials.project_id)

# 6. Prueba que funcione
print("Colecciones disponibles:", [c.id for c in client.collections()])

Colecciones disponibles: ['rutinarios']


In [12]:
db = firestore.Client(credentials=credentials, project=credentials.project_id)

In [13]:
# # 4. Define la ruta a tu documento anidado
# colec_raiz = "rutinarios"
# doc_proyecto = proyecto  # puede ser tu variable proyecto
# colec_valoriz = "presupuestos"
# id_valoriz = "desembolsos"

# doc_ref = (
#     db.collection(colec_raiz)
#     .document(doc_proyecto)
#     .collection(colec_valoriz)
#     .document(id_valoriz)
# )

# data_mantenimiento_res = doc_ref.get().to_dict()

# print(data_mantenimiento_res)

#### Documento del proyecto firebase

In [14]:
doc_proyecto_firebase = db.collection("rutinarios").document(proyecto).get().to_dict()

pprint.pprint(doc_proyecto_firebase)

{'contrato': {'contratista': {'razon_social': 'Grupo ARICOL E.I.R.L',
                              'ruc': 20606988398},
              'denominacion_tramo_convenio': 'EMP. AR-691 CALLA CALLA - '
                                             'YURACCANCHA - CUCHUHURI - '
                                             'PALLALLMAYO (KM 11+920)',
              'fecha_inicio': {'anio': 2025, 'dia': 16, 'mes': 4},
              'id_contrato': '006-2025',
              'jefe_mantenimiento': {'apellido': 'Tinta Cáceres',
                                     'dni': 0,
                                     'nombre': 'Genaro',
                                     'titulo': 'Ingeniero'},
              'monto_contrato': 59504.8,
              'numero_cuadrillas': 1,
              'numero_trabajadores': 3,
              'tiempo_ejecucion_dias': 240,
              'tipo_servicio': 'mantenimiento rutinario'},
 'datos_generales': {'distritos': ['Callalli'],
                     'provincia': 'Caylloma',
    

#### Contrato


In [15]:
contrato = doc_proyecto_firebase["contrato"]
pprint.pprint(contrato)

{'contratista': {'razon_social': 'Grupo ARICOL E.I.R.L', 'ruc': 20606988398},
 'denominacion_tramo_convenio': 'EMP. AR-691 CALLA CALLA - YURACCANCHA - '
                                'CUCHUHURI - PALLALLMAYO (KM 11+920)',
 'fecha_inicio': {'anio': 2025, 'dia': 16, 'mes': 4},
 'id_contrato': '006-2025',
 'jefe_mantenimiento': {'apellido': 'Tinta Cáceres',
                        'dni': 0,
                        'nombre': 'Genaro',
                        'titulo': 'Ingeniero'},
 'monto_contrato': 59504.8,
 'numero_cuadrillas': 1,
 'numero_trabajadores': 3,
 'tiempo_ejecucion_dias': 240,
 'tipo_servicio': 'mantenimiento rutinario'}


In [16]:
monto_contrato = contrato["monto_contrato"]
print(monto_contrato)

59504.8


#### Expediente técnico firebase

In [17]:
expediente_tecnico_firebase = doc_proyecto_firebase["expediente"]

pprint.pprint(expediente_tecnico_firebase)

{'cargas_trabajo': {'MR101': 7.95,
                    'MR102': 2558.43,
                    'MR103': 20.03,
                    'MR104': 71.2,
                    'MR203': 333.36,
                    'MR206': 237.37,
                    'MR301': 31230.64,
                    'MR401': 15.97,
                    'MR601': 39.73,
                    'MR701': 17.16},
 'codigo_ruta': 'AR-692',
 'coordenadas': {'fin': {'altitud': 4362,
                         'datum': 'WGS84',
                         'hemisferio': 'S',
                         'progresiva': 11920,
                         'x': 254510.31,
                         'y': 8277504.58,
                         'zona': None,
                         'zona_letra': None},
                 'inicio': {'altitud': 3980,
                            'datum': 'WGS84',
                            'hemisferio': 'S',
                            'progresiva': 0,
                            'x': 246384.65,
                            'y': 82800

#### Progresiva de inicio y fin del expediente

In [18]:
progresiva_inicio=expediente_tecnico_firebase["coordenadas"]["inicio"]["progresiva"]
progresiva_fin=expediente_tecnico_firebase["coordenadas"]["fin"]["progresiva"]

print(progresiva_inicio)
print(progresiva_fin)

0
11920


#### Valorización programada mensual


In [19]:
desembolsos =  db.collection("rutinarios").document(proyecto).collection('presupuestos').document('desembolsos').get().to_dict()
cronogramas_desembolsos = desembolsos['cronograma_desembolsos']
pprint.pprint(cronogramas_desembolsos)

{'10': {'gastos_operativos': 768.6,
        'igv': 1383.49,
        'mantenimiento_con_go': 8454.64,
        'mantenimiento_con_igv': 7686.04,
        'mantenimiento_sin_igv': 6302.55},
 '11': {'gastos_operativos': 743.81,
        'igv': 1338.86,
        'mantenimiento_con_go': 8181.91,
        'mantenimiento_con_igv': 7438.1,
        'mantenimiento_sin_igv': 6099.24},
 '12': {'gastos_operativos': 272.73,
        'igv': 490.91,
        'mantenimiento_con_go': 3000.02,
        'mantenimiento_con_igv': 2727.29,
        'mantenimiento_sin_igv': 2236.38},
 '4': {'gastos_operativos': 371.91,
       'igv': 669.43,
       'mantenimiento_con_go': 4090.96,
       'mantenimiento_con_igv': 3719.05,
       'mantenimiento_sin_igv': 3049.62},
 '5': {'gastos_operativos': 768.6,
       'igv': 1383.49,
       'mantenimiento_con_go': 8454.64,
       'mantenimiento_con_igv': 7686.04,
       'mantenimiento_sin_igv': 6302.55},
 '6': {'gastos_operativos': 743.81,
       'igv': 1338.86,
       'mantenimiento

In [20]:
desembolso_current_month = cronogramas_desembolsos[str(mes)]
print(desembolso_current_month["mantenimiento_con_igv"])

7686.04


#### Cargas de trabajo mensual presentadas por el contratista


In [21]:
valorizaciones = db.collection("rutinarios").document(proyecto).collection('valorizaciones').document(str(mes)).get().to_dict()
cargas_trabajo_contratista = valorizaciones['cargas_trabajo_contratista_corregido']
print(cargas_trabajo_contratista)

{'MR206': 33.91, 'MR203': 44.45, 'MR301': 3903.83, 'MR101': 0.99, 'MR104': 10.17, 'MR103': 5.01, 'MR102': 322.9, 'MR701': 3.43}


#### Cargas de trabajo del expediente técnico

In [22]:
cargas_trabajo_expediente_tecnico=doc_proyecto_firebase['expediente']['cargas_trabajo']

pprint.pprint(cargas_trabajo_expediente_tecnico)

{'MR101': 7.95,
 'MR102': 2558.43,
 'MR103': 20.03,
 'MR104': 71.2,
 'MR203': 333.36,
 'MR206': 237.37,
 'MR301': 31230.64,
 'MR401': 15.97,
 'MR601': 39.73,
 'MR701': 17.16}


#### Precios unitarios del expediente técnico

In [23]:
precios_unitarios_expediente_tecnico = doc_proyecto_firebase['expediente']['precios_unitarios']

pprint.pprint(precios_unitarios_expediente_tecnico)

{'MR101': 265.3,
 'MR102': 10.27,
 'MR103': 21.0,
 'MR104': 17.5,
 'MR201': 0.44,
 'MR202': 78.5,
 'MR203': 5.25,
 'MR204': 0.44,
 'MR205': 105,
 'MR206': 3.6,
 'MR301': 0.14,
 'MR401': 11.42,
 'MR501': 0.53,
 'MR601': 2.1,
 'MR701': 43.75,
 'MR702': 210}


In [24]:
# datos_cargas_trabajo_expediente_tecnico = {
#     "MR101": {"precio_unitario": 265.30, "carga_trabajo": 7.95},
#     "MR102": {"precio_unitario": 10.27, "carga_trabajo": 2558.43},
#     "MR103": {"precio_unitario": 21.0, "carga_trabajo": 20.03},
#     "MR104": {"precio_unitario": 17.50, "carga_trabajo": 71.20},
#     "MR203": {"precio_unitario": 5.25, "carga_trabajo": 333.36},
#     "MR206": {"precio_unitario": 3.6, "carga_trabajo": 237.37},
#     "MR301": {"precio_unitario": 0.14, "carga_trabajo": 31230.64},
#     "MR401": {"precio_unitario": 11.42, "carga_trabajo": 15.97},
#     "MR601": {"precio_unitario": 2.10, "carga_trabajo": 39.73},
#     "MR701": {"precio_unitario": 43.75, "carga_trabajo": 17.16},
# }

### Pickle

#### Cargas de trabajo programadas

In [25]:
# Concatenar la ruta completa al archivo .pkl
ruta_archivo = os.path.join("data",proyecto, f"{proyecto}_cargas_trabajo.pkl")

# Leer el archivo pickle
with open(ruta_archivo, "rb") as f:
    cargas_trabajo_programadas_anualmente = pickle.load(f)

cargas_trabajo_programadas_anualmente.tail(15)

,2025-04,2025-05,2025-06,2025-07,2025-08,2025-09,2025-10,2025-11,2025-12,TOTAL
codigo,,,,,,,,,,
MR103,0.000000,5.007500,0.000000,5.007500,5.007500,0.000000,5.007500,0.000000,0.000000,20.03
MR104,5.085714,10.171429,5.085714,10.171429,10.171429,5.085714,10.171429,10.171429,5.085714,71.20
MR201,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
MR202,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
MR203,22.224000,44.448000,22.224000,44.448000,44.448000,44.448000,44.448000,44.448000,22.224000,333.36
MR204,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
MR205,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
MR206,0.000000,33.910000,33.910000,33.910000,33.910000,33.910000,33.910000,33.910000,0.000000,237.37
MR301,1951.915000,3903.830000,3903.830000,3903.830000,3903.830000,3903.830000,3903.830000,3903.830000,1951.915000,31230.64


In [26]:
#eliminando el total
cargas_trabajo_programadas_anualmente = cargas_trabajo_programadas_anualmente.iloc[:-1]

In [27]:
cargas_trabajo_programadas_current_month = cargas_trabajo_programadas_anualmente[f"2025-{mes:02d}"].to_dict()
pprint.pprint(cargas_trabajo_programadas_current_month)

{'MR101': 0.99375,
 'MR102': 328.0038461538461,
 'MR103': 5.0075,
 'MR104': 10.17142857142857,
 'MR201': 0.0,
 'MR202': 0.0,
 'MR203': 44.448,
 'MR204': 0.0,
 'MR205': 0.0,
 'MR206': 33.91,
 'MR301': 3903.83,
 'MR401': 0.0,
 'MR501': 0.0,
 'MR601': 0.0,
 'MR701': 3.4320000000000004,
 'MR702': 0.0}


In [28]:
cargas_trabajo_programadas_anualmente.index.to_list()

['MR101',
 'MR102',
 'MR103',
 'MR104',
 'MR201',
 'MR202',
 'MR203',
 'MR204',
 'MR205',
 'MR206',
 'MR301',
 'MR401',
 'MR501',
 'MR601',
 'MR701',
 'MR702']

#### Cronograma anual

In [29]:
# Concatenar la ruta completa al archivo .pkl
ruta_archivo = os.path.join("data",proyecto, f"{proyecto}_cronograma_anual.pkl")

# Leer el archivo pickle
with open(ruta_archivo, "rb") as f:
    df_cronograma_anual = pickle.load(f)

df_cronograma_anual.head(15)

,2025-04,2025-05,2025-06,2025-07,2025-08,2025-09,2025-10,2025-11,2025-12,TOTAL
MR101,2,3,3,3,3,3,3,3,1,24
MR102,7,15,15,15,15,15,15,15,5,117
MR103,0,1,0,1,1,0,1,0,0,4
MR104,1,2,1,2,2,1,2,2,1,14
MR201,0,0,0,0,0,0,0,0,0,0
MR202,0,0,0,0,0,0,0,0,0,0
MR203,1,2,1,2,2,2,2,2,1,15
MR204,0,0,0,0,0,0,0,0,0,0
MR205,0,0,0,0,0,0,0,0,0,0
MR206,0,1,1,1,1,1,1,1,0,7


### JSON

#### Actividades

In [30]:
ruta_actividades= os.path.join("data", "general_data", "actividades.json")
with open(ruta_actividades, 'r', encoding='utf-8') as archivo:
    actividades = json.load(archivo)
# Ahora 'datos' es un diccionario de Python
print(actividades)

[{'key': 'MR100', 'value': {'label': 'Conservación de calzada', 'value': [{'key': 'MR101', 'value': {'label': 'Limpieza de calzada', 'carga_trabajo': 0, 'unidad': 'Km'}}, {'key': 'MR102', 'value': {'label': 'Bacheo', 'carga_trabajo': 0, 'unidad': 'm2'}}, {'key': 'MR103', 'value': {'label': 'Desquinche', 'carga_trabajo': 0, 'unidad': 'm3'}}, {'key': 'MR104', 'value': {'label': 'Remoción de derrumbes', 'carga_trabajo': 0, 'unidad': 'm3'}}]}}, {'key': 'MR200', 'value': {'label': 'Limpieza de obras de arte', 'value': [{'key': 'MR201', 'value': {'label': 'Limpieza de cunetas', 'carga_trabajo': 1440, 'unidad': 'm'}}, {'key': 'MR202', 'value': {'label': 'Limpieza de alcantarillas', 'carga_trabajo': 0, 'unidad': 'unidad'}}, {'key': 'MR203', 'value': {'label': 'Limpieza de badén', 'carga_trabajo': 0, 'unidad': 'm2'}}, {'key': 'MR204', 'value': {'label': 'Limpieza de zanjas de coronación', 'carga_trabajo': 0, 'unidad': 'm'}}, {'key': 'MR205', 'value': {'label': 'Limpieza de pontones', 'carga_tra

## Cálculos


### Cargas de trabajo y pu de expediente tecnico

In [31]:
datos_cargas_trabajo_expediente_tecnico=fusionar_diccionarios(
    {
        'precio_unitario':precios_unitarios_expediente_tecnico,
        'carga_trabajo':cargas_trabajo_expediente_tecnico
    }
)

pprint.pprint(datos_cargas_trabajo_expediente_tecnico)

{'MR101': {'carga_trabajo': 7.95, 'precio_unitario': 265.3},
 'MR102': {'carga_trabajo': 2558.43, 'precio_unitario': 10.27},
 'MR103': {'carga_trabajo': 20.03, 'precio_unitario': 21.0},
 'MR104': {'carga_trabajo': 71.2, 'precio_unitario': 17.5},
 'MR203': {'carga_trabajo': 333.36, 'precio_unitario': 5.25},
 'MR206': {'carga_trabajo': 237.37, 'precio_unitario': 3.6},
 'MR301': {'carga_trabajo': 31230.64, 'precio_unitario': 0.14},
 'MR401': {'carga_trabajo': 15.97, 'precio_unitario': 11.42},
 'MR601': {'carga_trabajo': 39.73, 'precio_unitario': 2.1},
 'MR701': {'carga_trabajo': 17.16, 'precio_unitario': 43.75}}


In [32]:
# Crear DataFrame usando pd.DataFrame.from_dict() con orient='index'
df_cargas_trabajo_expediente = pd.DataFrame.from_dict(
    datos_cargas_trabajo_expediente_tecnico, orient="index"
)

# Resetear el índice para convertir las claves MR en una columna
df_cargas_trabajo_expediente = df_cargas_trabajo_expediente.reset_index().rename(
    columns={"index": "codigo_MR"}
)

In [33]:
# Agregar columna parcial
df_cargas_trabajo_expediente["parcial"] = (
    df_cargas_trabajo_expediente["precio_unitario"]
    * df_cargas_trabajo_expediente["carga_trabajo"]
)

In [34]:
df_cargas_trabajo_expediente.head()

,codigo_MR,precio_unitario,carga_trabajo,parcial
0,MR104,17.50,71.20,1246.000
1,MR601,2.10,39.73,83.433
2,MR203,5.25,333.36,1750.140
3,MR103,21.00,20.03,420.630
4,MR101,265.30,7.95,2109.135


In [35]:
# Calcular el total
costo_directo = df_cargas_trabajo_expediente["parcial"].sum()
print("costo_directo", costo_directo)

costo_directo 38044.3631


In [36]:
costo_total_expediente = calculadora_costo_total(costo_directo)
print(costo_total_expediente["total"])

51626.2


In [37]:
# Agregar columna parcial
df_cargas_trabajo_expediente["precio_unitario_actualizado"] = (
    df_cargas_trabajo_expediente["precio_unitario"]
    * (monto_contrato / costo_total_expediente["total"])
)

df_cargas_trabajo_expediente["parcial_actualizado"] = (
    df_cargas_trabajo_expediente["precio_unitario_actualizado"]
    * df_cargas_trabajo_expediente["carga_trabajo"]
)

# Calcular el total
costo_directo_actualizado = df_cargas_trabajo_expediente["parcial_actualizado"].sum()

print(f"El costo directo es: {costo_directo_actualizado}")

El costo directo es: 43850.258539130904


In [38]:
df_cargas_trabajo_expediente.head(10)

,codigo_MR,precio_unitario,carga_trabajo,parcial,precio_unitario_actualizado,parcial_actualizado
0,MR104,17.50,71.20,1246.0000,20.170650,1436.150265
1,MR601,2.10,39.73,83.4330,2.420478,96.165590
2,MR203,5.25,333.36,1750.1400,6.051195,2017.226344
3,MR103,21.00,20.03,420.6300,24.204780,484.821738
4,MR101,265.30,7.95,2109.1350,305.787051,2431.007054
5,MR401,11.42,15.97,182.3774,13.162790,210.209752
6,MR301,0.14,31230.64,4372.2896,0.161365,5039.538416
7,MR102,10.27,2558.43,26275.0761,11.837290,30284.877607
8,MR701,43.75,17.16,750.7500,50.426624,865.320876
9,MR206,3.60,237.37,854.5320,4.149391,984.940897


In [39]:
pago_costo_total_contratista = calculadora_costo_total(costo_directo_actualizado)
print(pago_costo_total_contratista["total"])

59504.8


In [40]:
dict_precios_unitarios_actualizados = dict(
    zip(
        df_cargas_trabajo_expediente["codigo_MR"],
        df_cargas_trabajo_expediente["precio_unitario_actualizado"],
    )
)

pprint.pprint(dict_precios_unitarios_actualizados)

{'MR101': 305.78705076104774,
 'MR102': 11.837289903188692,
 'MR103': 24.204779743618552,
 'MR104': 20.170649786348793,
 'MR203': 6.051194935904638,
 'MR206': 4.1493908131917525,
 'MR301': 0.16136519829079038,
 'MR401': 13.162789746291613,
 'MR601': 2.4204779743618556,
 'MR701': 50.42662446587199}


### Cálculo de pago de acuerdo a cargas de trabajo


In [41]:
pago_costo_directo_parciales_contratista = calcular_costos_unitarios(
    dict_precios_unitarios_actualizados, cargas_trabajo_contratista
)
pprint.pprint(pago_costo_directo_parciales_contratista)

{'MR101': 302.7291802534373,
 'MR102': 3822.2609097396285,
 'MR103': 121.26594651552894,
 'MR104': 205.13550832716723,
 'MR203': 268.97561490096115,
 'MR206': 140.7058424753323,
 'MR301': 629.9423020435362,
 'MR701': 172.96332191794093}


##### Visualizacion en dataframe


In [42]:
df_pago_costo_directo_parciales_contratista = pd.DataFrame.from_dict(
    pago_costo_directo_parciales_contratista, orient="index"
)

df_pago_costo_directo_parciales_contratista = (
    df_pago_costo_directo_parciales_contratista.reset_index().rename(
        columns={"index": "codigo_MR"}
    )
)

df_pago_costo_directo_parciales_contratista = (
    df_pago_costo_directo_parciales_contratista.rename(columns={0: "monto_pago"})
)

df_pago_costo_directo_parciales_contratista

,codigo_MR,monto_pago
0,MR206,140.705842
1,MR203,268.975615
2,MR301,629.942302
3,MR101,302.729180
4,MR104,205.135508
5,MR103,121.265947
6,MR102,3822.260910
7,MR701,172.963322


In [43]:
# Ordenar el DataFrame por la columna 'monto_pago'
# Por defecto, el orden es ascendente (de menor a mayor)
df_ordenado = df_pago_costo_directo_parciales_contratista.sort_values(
    by="monto_pago", ascending=False
)

print("\nDataFrame Ordenado por 'monto_pago' (ascendente):")
print(df_ordenado)


DataFrame Ordenado por 'monto_pago' (ascendente):
  codigo_MR   monto_pago
6     MR102  3822.260910
2     MR301   629.942302
3     MR101   302.729180
1     MR203   268.975615
4     MR104   205.135508
7     MR701   172.963322
0     MR206   140.705842
5     MR103   121.265947


In [44]:
# Lambda que suma todos los valores de un diccionario
sumar_valores = lambda d: sum(d.values())

In [45]:
pago_costo_directo_contratista = sumar_valores(pago_costo_directo_parciales_contratista)
print(pago_costo_directo_contratista)

5663.978626173533


In [46]:
pago_costo_total_contratista = calculadora_costo_total(pago_costo_directo_contratista)
print(pago_costo_total_contratista["total"])

7686.02


##### Validación


In [47]:
diferencia_costos = abs(
    pago_costo_total_contratista["total"]
    - desembolso_current_month["mantenimiento_con_igv"]
)

pprint.pprint(
    {
        "diferencia_costos": diferencia_costos,
        "ejecutado": pago_costo_total_contratista["total"],
        "programado": desembolso_current_month["mantenimiento_con_igv"],
    }
)

if diferencia_costos > 1:
    raise ValueError(
        "La valorizacion de las actividades presentadas por el contratista no es coherente con lo programado"
    )

{'diferencia_costos': 0.019999999999527063,
 'ejecutado': 7686.02,
 'programado': 7686.04}


### Ordenando cargas de trabajo

In [48]:
cargas_trabajo_ordenadas=ordenar_por_centena(cargas_trabajo_programadas_current_month)
pprint.pprint(cargas_trabajo_ordenadas)

[{'key': 'MR101', 'value': 0.99375},
 {'key': 'MR102', 'value': 328.0038461538461},
 {'key': 'MR103', 'value': 5.0075},
 {'key': 'MR104', 'value': 10.17142857142857},
 {'key': 'MR201', 'value': 0.0},
 {'key': 'MR202', 'value': 0.0},
 {'key': 'MR203', 'value': 44.448},
 {'key': 'MR204', 'value': 0.0},
 {'key': 'MR205', 'value': 0.0},
 {'key': 'MR206', 'value': 33.91},
 {'key': 'MR301', 'value': 3903.83},
 {'key': 'MR401', 'value': 0.0},
 {'key': 'MR501', 'value': 0.0},
 {'key': 'MR601', 'value': 0.0},
 {'key': 'MR701', 'value': 3.4320000000000004},
 {'key': 'MR702', 'value': 0.0}]


## Escribiendo el excel

### Path

In [49]:
# Ruta dinámica
ruta_directorio = os.path.join("output", proyecto, str(mes))
ruta_archivo = os.path.join(ruta_directorio, "cargas_trabajo.xlsx")
# Crear carpetas si no existen
os.makedirs(ruta_directorio, exist_ok=True)

wb = xlsxwriter.Workbook(ruta_archivo)
ws=wb.add_worksheet("cargas_trabajo")

### Formatos

In [50]:
# 1. Propiedades base
BASE_CELL = {
    "valign": "vcenter",
    "border": 1,
}

BASE_HEADER = {
    **BASE_CELL,
    "bold": True,
}

BG_HEADER = "#D9E1F2"
BG_TABLE = "#C5D9F1"


# 2. Función de helper para crear formatos
def fmt(wb, **props):
    cfg = {}
    # parte genérica
    cfg.update(props)
    return wb.add_format(cfg)


# 3. Diccionario de formatos
formats = {
    # encabezados
    "header": {"align": "center", "bg_color": BG_HEADER, **BASE_HEADER},
    "header2": {"align": "left", **BASE_HEADER},
    "header_label": {"align": "center", **BASE_HEADER},
    # formato vertical header
    "header_vertical": {
        **BASE_HEADER,
        "bold": True,
        "align": "center",
        "valign": "vcenter",
        "bg_color": BG_HEADER,
        "rotation": 90,  # <— ¡aquí está el giro!
    },
    # encabezados de tabla
    "table_header": {
        "align": "center",
        "text_wrap": True,
        "shrink": True,
        "bg_color": BG_TABLE,
        **BASE_CELL,
    },
    "table_header_int": {"align": "center", "bg_color": BG_TABLE, **BASE_CELL},
    # celdas numéricas y de texto
    "cell": {"font_size": 8, "num_format": "#,##0.00", **BASE_CELL},
    # "cell": {"font_size": 8, "num_format": "#,##0.00", **BASE_CELL},
    "cell_int": {"num_format": "#,##0", **BASE_CELL},
    "cell_number": {"num_format": "#,##0.00", **BASE_CELL},
    "cell_number_percent": {
        "num_format": "#,##0.00%",
        "align": "right",
        **BASE_CELL,
    },
    "cell_text": {"align": "left", "text_wrap": True, "shrink": True, **BASE_CELL},
    "cell_right": {"align": "right", **BASE_CELL},
    # sumas
    "suma": {"num_format": "#,##0.00", **BASE_CELL},
    # soles
    "soles_color": {
        "bold": True,
        "bg_color": BG_HEADER,
        "num_format": '"S/." #,##0.00',
        **BASE_CELL,
    },
    "soles": {"num_format": '"S/." #,##0.00', **BASE_CELL},
    # formato de ajuste general
    "ajustar": {"align": "center", "text_wrap": True, "shrink": True, **BASE_CELL},
    "only_border": {
        "border": 1,
    },
}

# 4. Creación dinámica de los objetos Format
wb_formats = {name: fmt(wb, **props) for name, props in formats.items()}

# Ahora sólo usa wb_formats["header"], wb_formats["cell_text"], etc.

In [51]:
# Punto de inserción (cero-indexed)
table_start_row = 1   # si quieres que empiece en la fila 7
table_start_col = 1   # si quieres que empiece en la columna B

### Escritores de excel

In [52]:
def merge(ws, r1, c1, r2, c2, valor, fmt):
    """Merge relative a table_start_* + offsets."""
    ws.merge_range(
        xl_range(table_start_row + r1,
                 table_start_col + c1,
                 table_start_row + r2,
                 table_start_col + c2),
        valor, fmt
    )

In [53]:
def write_rel(ws, row_offset, col_offset, value, fmt=None):
    """
    Escribe en la celda (table_start_row + row_offset, table_start_col + col_offset)
    """
    abs_row = table_start_row + row_offset
    abs_col = table_start_col + col_offset
    ws.write(abs_row, abs_col, value, fmt)

In [54]:
def merge_xy(ws, start: tuple[int,int], end: tuple[int,int], value, fmt):
    """
    Merge de celdas desde start=(x1, y1) hasta end=(x2, y2),
    relativas a table_start_col/row.
    """
    (x1, y1), (x2, y2) = start, end
    ws.merge_range(
        xl_range(
            table_start_row + y1,  # row1 absoluto
            table_start_col + x1,  # col1 absoluto
            table_start_row + y2,  # row2 absoluto
            table_start_col + x2,  # col2 absoluto
        ),
        value, fmt
    )

def write_xy(ws, pos: tuple[int,int], value, fmt=None):
    """
    Escribe un valor en pos=(x, y) relativa a table_start_col/row.
    """
    x, y = pos
    ws.write(
        table_start_row + y,
        table_start_col + x,
        value, fmt
    )

In [55]:
def set_columns_rel(ws, rel_col_widths):
    """
    Aplica ws.set_column a rangos de columnas definidos
    en coordenadas relativas (x) a table_start_col.
    rel_col_widths: lista de tuplas (first_rel, last_rel, width)
    """
    for first_rel, last_rel, width in rel_col_widths:
        abs_first = table_start_col + first_rel
        abs_last  = table_start_col + last_rel
        ws.set_column(abs_first, abs_last, width)

### Anchos de columnas

In [56]:
col_widths_rel = [
    (0, 0, 8),  # columna B en adelante:  B–B ancho=8
    (1, 1, 12),  # C–C ancho=30
    (2, 2, 30),  # D–H ancho=8
    (3, 3, 12),  # I–T ancho=8
    (4, 4, 10),  # U–U ancho=10
    (5, 5, 10),  # U–U ancho=10
    (6, 6, 14),  # U–U ancho=10
    (7, 7, 14),
]

set_columns_rel(ws, col_widths_rel)

### Comenzando a escribir el excel

#### Encabezados de la tabla

In [57]:
merge_xy(ws, (0, 0), (0, 1), "N.º", wb_formats["header"])
merge_xy(ws, (1, 0), (1, 1), "Código", wb_formats["header"])
merge_xy(ws, (2, 0), (2, 1), "Actividades", wb_formats["header"])

merge_xy(ws, (3, 0), (3, 1), "Unidad", wb_formats["header"])

merge_xy(ws, (4, 0), (5, 0), "Cargas de trabajo", wb_formats["header"])

write_xy(ws, (4, 1), "Programación", wb_formats["header_vertical"])
write_xy(ws, (5, 1), "Ejecución", wb_formats["header_vertical"])

merge_xy(ws, (6, 0), (6, 1), "Avance", wb_formats["header"])
merge_xy(ws, (7, 0), (7, 1), "Progresivas", wb_formats["header"])

#### Contenido de la tabla:

In [58]:
current_col = 0
current_row = 2

In [59]:
columnas_insertadas = 0

impresos = set()

contador_items = 1

for grupo, elementos in itertools.groupby(cargas_trabajo_ordenadas, clave_grupo):
    current_col = 0

    # codigo=int(carga_trabajo['key'][2:])
    # grupo=codigo//100
    # print(grupo)

    inicio = grupo * 100
    fin = inicio + 99

    print(f"Valores entre {inicio} al {fin}!")
    resultado = next(
        (item for item in actividades if item["key"] == f"MR{inicio}"), None
    )
    # print(resultado)

    write_xy(ws, (current_col, current_row), "", wb_formats["cell_text"])  # numero
    current_col += 1
    write_xy(
        ws, (current_col, current_row), resultado["key"], wb_formats["cell_text"]
    )  # codigo
    current_col += 1
    write_xy(
        ws,
        (current_col, current_row),
        resultado["value"]["label"],
        wb_formats["cell_text"],
    )  #

    current_row += 1

    for it in elementos:
        current_col = 0
        # print(f"  {it['key']}: {it['value']}")
        print(f"  {it['key']}: {it['value']}")

        resultado2 = next(
            (item for item in resultado["value"]["value"] if item["key"] == it["key"]),
            None,
        )
        print(resultado)

        write_xy(
            ws, (current_col, current_row), contador_items, wb_formats["cell_int"]
        )  # numero
        current_col += 1
        write_xy(
            ws, (current_col, current_row), resultado2["key"], wb_formats["cell_text"]
        )
        current_col += 1
        write_xy(
            ws,
            (current_col, current_row),
            resultado2["value"]["label"],
            wb_formats["cell_text"],
        )
        current_col += 1
        write_xy(
            ws,
            (current_col, current_row),
            resultado2["value"]["unidad"],
            wb_formats["cell_text"],
        )
        current_col += 1
        write_xy(
            ws,
            (current_col, current_row),
            cargas_trabajo_programadas_anualmente.loc[it["key"], f"{anio}-{mes:02d}"],
            wb_formats["cell_number"],
        )
        current_col += 1
        write_xy(
            ws,
            (current_col, current_row),
            cargas_trabajo_contratista.get(it["key"],0),
            wb_formats["cell_number"],
        )
        current_col += 1

        avance = calcular_avance(
            programado=cargas_trabajo_programadas_anualmente.loc[it["key"], f"{anio}-{mes:02d}"],
            ejecutado=cargas_trabajo_contratista.get(it["key"],0),
        )
        write_xy(
            ws,
            (current_col, current_row),
            avance,
            wb_formats["cell_number_percent"],
        )
        current_col += 1

        progresivas = (
            formatear_progresiva(progresiva_inicio)
            + " - "
            + formatear_progresiva(progresiva_fin)
        )
        write_xy(
            ws,
            (current_col, current_row),
            progresivas,
            wb_formats["cell_number_percent"],
        )
        current_col += 1

        #######3
        contador_items += 1
        current_row += 1

Valores entre 100 al 199!
  MR101: 0.99375
{'key': 'MR100', 'value': {'label': 'Conservación de calzada', 'value': [{'key': 'MR101', 'value': {'label': 'Limpieza de calzada', 'carga_trabajo': 0, 'unidad': 'Km'}}, {'key': 'MR102', 'value': {'label': 'Bacheo', 'carga_trabajo': 0, 'unidad': 'm2'}}, {'key': 'MR103', 'value': {'label': 'Desquinche', 'carga_trabajo': 0, 'unidad': 'm3'}}, {'key': 'MR104', 'value': {'label': 'Remoción de derrumbes', 'carga_trabajo': 0, 'unidad': 'm3'}}]}}
  MR102: 328.0038461538461
{'key': 'MR100', 'value': {'label': 'Conservación de calzada', 'value': [{'key': 'MR101', 'value': {'label': 'Limpieza de calzada', 'carga_trabajo': 0, 'unidad': 'Km'}}, {'key': 'MR102', 'value': {'label': 'Bacheo', 'carga_trabajo': 0, 'unidad': 'm2'}}, {'key': 'MR103', 'value': {'label': 'Desquinche', 'carga_trabajo': 0, 'unidad': 'm3'}}, {'key': 'MR104', 'value': {'label': 'Remoción de derrumbes', 'carga_trabajo': 0, 'unidad': 'm3'}}]}}
  MR103: 5.0075
{'key': 'MR100', 'value': {'

In [60]:
print(current_row, current_col)

25 8


In [61]:
print(
    xl_range(
        table_start_row,
        table_start_col,
        table_start_row + current_row-1,
        table_start_col+current_col-1,
    ),
)

B2:I26


In [62]:
# # colocar grilla  a toda la tabla
# ws.conditional_format(
#     xl_range(
#         table_start_row,
#         table_start_col,
#         table_start_row + current_row - 1,
#         table_start_col + current_col - 1,
#     ),
#     {"type": "no_errors", "format": wb_formats["only_border"]},
# )

In [63]:
# Función más robusta que aplica bordes sin afectar el contenido
# def add_borders_to_existing_range(ws, wb, start_row, start_col, end_row, end_col):
#     # Crear formato base con bordes
#     for row in range(start_row, end_row + 1):
#         for col in range(start_col, end_col + 1):
#             # Usar write_blank para celdas que solo necesitan formato
#             ws.write_blank(row, col, wb_formats["only_border"])

# # Llamar después de escribir todos tus datos
# add_borders_to_existing_range(
#     ws, wb,
#     table_start_row,
#     table_start_col,
#     table_start_row + current_row - 1,
#     table_start_col + current_col - 1
# )

In [64]:
# Guardar archivo
wb.close()

In [65]:
ruta_archivo = Path(ruta_archivo).resolve()

In [66]:
# Aplicar bordes con Excel
excel = win32.Dispatch("Excel.Application")
excel.Visible = False

workbook = excel.Workbooks.Open(ruta_archivo)
worksheet = workbook.ActiveSheet

# Seleccionar tu rango
range_obj = worksheet.Range(
    worksheet.Cells(table_start_row + 1, table_start_col + 1),
    worksheet.Cells(table_start_row + current_row, table_start_col + current_col)
)

# Aplicar todos los bordes
range_obj.Borders.LineStyle = 1
range_obj.Borders.Weight = 2

workbook.Save()
workbook.Close()
excel.Quit()


### Imprimiendo una imagen a partir del excel

In [67]:
# 1. Carga el libro y la hoja

wb = load_workbook(ruta_archivo, data_only=True)
ws = wb.active  # o wb["NombreHoja"]

# 2. Determina el rango usado
min_row, min_col = ws.min_row, ws.min_column
max_row, max_col = ws.max_row, ws.max_column
rango = f"{get_column_letter(min_col)}{min_row}:{get_column_letter(max_col)}{max_row}"

# 3. Exporta a PNG
salida_png = f"cargas_trabajo.png"
ruta_salida_image= os.path.join(ruta_directorio, salida_png)
excel2img.export_img(ruta_archivo, ruta_salida_image, ws.title, rango)

print(f"Imagen generada: {salida_png}")

Imagen generada: cargas_trabajo.png
